In [1]:
cd /home/ubuntu/DistMEC/

/home/ubuntu/DistMEC


In [2]:
import numpy as np
import matplotlib.pyplot as plt

from classes.User import *
from classes.solver import *
from classes.Server import *

In [32]:
def gen_eq_locs(space_1d, nums, offset = 0.5):
    # Generate well spread out locations in square space
    num_across = int(np.floor(np.sqrt(nums)))
    locs = []
    
    inc = space_1d/nums
    
    for i,j in itertools.product(range(num_across), range(num_across)):
        locs += [(i*inc+offset, j*inc+offset)]
    
    return locs

def obtain_w(Users, num_users, num_svrs):
    
    w_curr = np.zeros([num_users,num_svrs])
    for i in range(num_users):
        w_curr[i] = Users[i].reward_scale[Users[i].usr_place]
    
    return w_curr

def update_user_locs(Users):
    
    for i in range(len(Users)):
        Users[i].next_loc()
    return
    
def get_arms_list(Users):
    arms = []
    for i in range(len(Users)):
        arms+= [Users[i].choose_arm()]
    return arms

def sort_server_results(arms_list, Servers, Users):
    
    scales_dict = {}
    usr_idxs_dict = {}
    w_est_dict = {}
    
    
    for s in range(len(Servers)):
        usr_idxs = np.argwhere(arms_list == s).flatten()
        scales = np.zeros(usr_idxs.shape[0])
        w_est = np.zeros(usr_idxs.shape[0])
        stay_times = np.zeros()
        for u in range(usr_idxs.shape[0])
            scales[u] = Users[usr_idxs[u]].reward_scale[Users[usr_idxs[u]].usr_place]
            w_est[u] =  Users[usr_idxs[u]].ucb_raw[s]
        scales_dict[s] = copy.deepcopy(scales)
        usr_idxs_dict[s] = copy.deepcopy(usr_idxs)
        w_est_dict[s] = copy.deepcopy(w_est)
        
        Servers[s].receive_users(user_list, scales_list, w_est_list, stay_times_list)
    
    
    return scales_dict, usr_idxs_dict, w_est_dict


def update_user_info(Users):
    return

In [33]:
# System Parameters
T = 10000
num_users = 10
num_svrs = 16
num_locs = 16
mu = np.random.uniform(low=0, high = 1, size = [num_users, num_svrs])
# w = np.random.uniform(low=0, high=1, size= [num_users, num_svrs])

# Generate visitng locations and server locations
space_1d_dim = 16
usr_locs = gen_eq_locs(space_1d=space_1d_dim, nums=num_locs)
svr_locs = gen_eq_locs(space_1d=space_1d_dim, nums=num_svrs)

# Create Users
Users = []
for i in range(num_users):
    Users += [User(usr_locs,svr_locs,mu[i],i)]
    
# Create Servers
Servers = []
for i in range(num_svrs):
    Servers += [Server(svr_locs[i],mu,i)]

# Recorded Values
rewards_record = np.zeros([num_users,num_svrs])
pulls_record = np.zeros([num_users,num_svrs])
ucb = np.zeros([num_users,num_svrs])
regret = np.zeros(T)

In [40]:
a = np.array([1,2,3,4])
b = np.argwhere(a==5)
b.flatten()
b.shape

(0, 1)

In [ ]:
# Initial Stages -- TS = servers, rotating 
arms = list(range(num_users)) 

for i in range(num_svrs):
    w = obtain_w(Users)
    optimal = offline_optimal_action(w, mu)
    rewards_record, pulls_record = pull_super_arm(arms, mu, rewards_record, pulls_record)
    regret[i] = optimal[1] - expected_reward(arms, mu, w)
    arms = sweep_init_next(arms, num_svrs)
    
for t in range(i+1, T):
    w = np.random.uniform(low=0, high=1, size= [num_users, num_svrs])
    optimal = offline_optimal_action(w, mu)
    ucb = update_ucb(rewards_record, pulls_record, ucb, i, 1)
    arms = offline_optimal_action(w, ucb)[0]
    rewards_record, pulls_record = pull_super_arm(arms, mu, rewards_record, pulls_record)
    regret[t] = optimal[1] - expected_reward(arms, mu, w)